## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [1]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "default" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-54-166-7-244.compute-1.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-54-166-7-244.compute-1.amazonaws.com:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://totaleren-mlflow-artifacts-remote/1', creation_time=1685110833154, experiment_id='1', last_update_time=1685110833154, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://totaleren-mlflow-artifacts-remote/0', creation_time=1685110509350, experiment_id='0', last_update_time=1685110509350, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: 's3://totaleren-mlflow-artifacts-remote/1/435aad32e09c467499658aea79dc07a8/artifacts'


In [23]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://totaleren-mlflow-artifacts-remote/1', creation_time=1685110833154, experiment_id='1', last_update_time=1685110833154, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://totaleren-mlflow-artifacts-remote/0', creation_time=1685110509350, experiment_id='0', last_update_time=1685110509350, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [24]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1685111378199, description='', last_updated_timestamp=1685111378868, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1685111378868, current_stage='None', description='', last_updated_timestamp=1685111378868, name='iris-classifier', run_id='435aad32e09c467499658aea79dc07a8', run_link='', source='s3://totaleren-mlflow-artifacts-remote/1/435aad32e09c467499658aea79dc07a8/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]

In [25]:
runs = mlflow.search_runs(experiment_ids='1')
run_id = runs.loc[0,'run_id']
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2023/05/26 11:31:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 2
Created version '2' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1685111498575, current_stage='None', description='', last_updated_timestamp=1685111498575, name='iris-classifier', run_id='435aad32e09c467499658aea79dc07a8', run_link='', source='s3://totaleren-mlflow-artifacts-remote/1/435aad32e09c467499658aea79dc07a8/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='2'>